In [4]:
from transformers import BertTokenizer, T5ForConditionalGeneration, Text2TextGenerationPipeline


加载模型和tokenizer

In [5]:
tokenizer = BertTokenizer.from_pretrained("../uer/t5-base-chinese-cluecorpussmall")
model = T5ForConditionalGeneration.from_pretrained("../uer/t5-base-chinese-cluecorpussmall")

d:\Programs\miniconda3\envs\llm\lib\site-packages\transformers\modeling_utils.py:442: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_lo

加载数据集

In [6]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="../data/DuReaderQG/train.json", split="train")
datasets = dataset.train_test_split(test_size=0.2)
dev_dataset = load_dataset("json", data_files="../data/DuReaderQG/dev.json", split="train")
datasets, dev_dataset

(DatasetDict({
     train: Dataset({
         features: ['context', 'answer', 'question', 'id'],
         num_rows: 11616
     })
     test: Dataset({
         features: ['context', 'answer', 'question', 'id'],
         num_rows: 2904
     })
 }),
 Dataset({
     features: ['context', 'answer', 'question', 'id'],
     num_rows: 984
 }))

In [7]:
tokenize_example = tokenizer("context: " + datasets["train"][0]["context"] + "question: " + datasets["train"][0]["question"], return_tensors="pt", max_length=256, truncation=True, padding="max_length")
tokenize_example

{'input_ids': tensor([[  101, 11485, 11816,   131,   671,  5663,  4638,  8202,  1914,   671,
          1417,  8024,  6574,  7030,  6820,  1377,   809,   100,   118,   100,
           118,   100,  1798,  3382,  2595,  3300,  3322,  1843,  3160,  3221,
          2443,  1773,  2128,   830,  1062,  1385,  3418,  2945,   100,   118,
          8170,  3403,  1114,  4777,  1169,  2768,  1216,  4638,  3173,  1798,
           772,  1501,  8024,  1072,  3300,  5447,  4125,  3353,  7361,  7770,
           510,  1355,  4170,  7030,   856,  5023,  4294,  4157,   511,  6421,
           772,  1501,  6820,  1072,  3300,  5679,  1962,  4638,  1377,  1848,
          2595,  8024,  5447,  7000,   510,  5447,  4822,   510,  5447,  5576,
          6008,  2595,  8024,  3177,  2339,   510,  5335,   934,  3683,  6772,
          3175,   912,  8024,  2400,  5543,   680,  1071,   800,  2196,  1843,
          3332,  3160,  8020,  1963,  7344,  4125,  1259,  5023,  8021,  1394,
          2400,  2418,  4500,   511,  

In [8]:
label = tokenizer(datasets["train"][0]["answer"], max_length=64, truncation=True, padding="max_length")
label["input_ids"][label["input_ids"] == tokenizer.pad_token_id] = -100
label

{'input_ids': [-100, 8202, 1914, 671, 1417, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [9]:
tokenize_example["labels"] = label["input_ids"]
tokenize_example

{'input_ids': tensor([[  101, 11485, 11816,   131,   671,  5663,  4638,  8202,  1914,   671,
          1417,  8024,  6574,  7030,  6820,  1377,   809,   100,   118,   100,
           118,   100,  1798,  3382,  2595,  3300,  3322,  1843,  3160,  3221,
          2443,  1773,  2128,   830,  1062,  1385,  3418,  2945,   100,   118,
          8170,  3403,  1114,  4777,  1169,  2768,  1216,  4638,  3173,  1798,
           772,  1501,  8024,  1072,  3300,  5447,  4125,  3353,  7361,  7770,
           510,  1355,  4170,  7030,   856,  5023,  4294,  4157,   511,  6421,
           772,  1501,  6820,  1072,  3300,  5679,  1962,  4638,  1377,  1848,
          2595,  8024,  5447,  7000,   510,  5447,  4822,   510,  5447,  5576,
          6008,  2595,  8024,  3177,  2339,   510,  5335,   934,  3683,  6772,
          3175,   912,  8024,  2400,  5543,   680,  1071,   800,  2196,  1843,
          3332,  3160,  8020,  1963,  7344,  4125,  1259,  5023,  8021,  1394,
          2400,  2418,  4500,   511,  

数据预处理

In [10]:
def process_function(examples):
    context_questions = [f"context:{context} question:{question}" for context, question in zip(examples["context"], examples["question"])]
    tokenized_examples = tokenizer(context_questions, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(examples["answer"], max_length=64, truncation=True, padding="max_length")
    labels["input_ids"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in label]
        for label in labels["input_ids"]
    ]
    tokenized_examples["labels"] = labels["input_ids"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

Map:   0%|          | 0/11616 [00:00<?, ? examples/s]

Map:   0%|          | 0/2904 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 11616
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2904
    })
})

In [11]:
tokenized_datasets["train"][0]

{'input_ids': [101,
  11485,
  11816,
  131,
  671,
  5663,
  4638,
  8202,
  1914,
  671,
  1417,
  8024,
  6574,
  7030,
  6820,
  1377,
  809,
  100,
  118,
  100,
  118,
  100,
  1798,
  3382,
  2595,
  3300,
  3322,
  1843,
  3160,
  3221,
  2443,
  1773,
  2128,
  830,
  1062,
  1385,
  3418,
  2945,
  100,
  118,
  8170,
  3403,
  1114,
  4777,
  1169,
  2768,
  1216,
  4638,
  3173,
  1798,
  772,
  1501,
  8024,
  1072,
  3300,
  5447,
  4125,
  3353,
  7361,
  7770,
  510,
  1355,
  4170,
  7030,
  856,
  5023,
  4294,
  4157,
  511,
  6421,
  772,
  1501,
  6820,
  1072,
  3300,
  5679,
  1962,
  4638,
  1377,
  1848,
  2595,
  8024,
  5447,
  7000,
  510,
  5447,
  4822,
  510,
  5447,
  5576,
  6008,
  2595,
  8024,
  3177,
  2339,
  510,
  5335,
  934,
  3683,
  6772,
  3175,
  912,
  8024,
  2400,
  5543,
  680,
  1071,
  800,
  2196,
  1843,
  3332,
  3160,
  8020,
  1963,
  7344,
  4125,
  1259,
  5023,
  8021,
  1394,
  2400,
  2418,
  4500,
  511,
  100,
  118,
  100

In [12]:
from transformers import TrainingArguments

train_args = TrainingArguments(
    output_dir="./output/generateAskAnswer",
    num_train_epochs=20,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    weight_decay=0.01,
    warmup_steps=100,
)

In [14]:
from transformers import Trainer, DefaultDataCollator

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=DefaultDataCollator(),
    tokenizer=tokenizer,
)

In [15]:
trainer.train()

d:\Programs\miniconda3\envs\llm\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/7260 [00:00<?, ?it/s]

{'loss': 9.2797, 'learning_rate': 1e-05, 'epoch': 0.03}
{'loss': 9.2476, 'learning_rate': 2e-05, 'epoch': 0.06}
{'loss': 9.1917, 'learning_rate': 3e-05, 'epoch': 0.08}


KeyboardInterrupt: 

In [ ]:
text2text_generator = Text2TextGenerationPipeline(model, tokenizer)  
text2text_generator("中国的首都是extra0京", max_length=50, do_sample=False)

[{'generated_text': 'extra0 北 extra1 extra2 extra3 extra4 extra5 extra6 extra7 extra8 extra9 extra10 extra11 extra12 extra13 extra14 extra15 extra16 extra17 extra18 extra19 extra9'}]